<a href="https://colab.research.google.com/github/uttasarga9067/imdb_movie_review_classification_model/blob/main/ut_imdb_movies_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:

from huggingface_hub import notebook_login

# Login to Huggingface to get access to the model
notebook_login()

In [1]:
%pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.1 MB/s eta 0:00:00


In [2]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.3 MB/s eta 0:00:00


In [3]:
!pip install accelerate

In [4]:
from datasets import load_dataset
import os
import pandas as pd
import transformers as tr

In [5]:
imbd_ds = load_dataset("imdb", ignore_verifications=True)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2491: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
model_checkpoint = "t5-small"
tokenizer = tr.AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [7]:
def to_tokens(
    tokenizer: tr.models.t5.tokenization_t5_fast.T5TokenizerFast,
    label_map:dict) -> callable:
    """
    Given a 'tokenizer' this closure will iterate through x and return the result of apply()
    This function is then mapped to the dataset and returned with ids and attention mask.
    """

    def apply(x) -> tr.tokenization_utils_base.BatchEncoding:
      """
      From a formatted dataset 'x', a batch encoding 'token_res' is created
      """
      target_labels = [label_map[y] for y in x['label']]
      token_res = tokenizer(
        x['text'],
        text_target = target_labels,
        return_tensors = 'pt',
        truncation =True,
        padding=True,
      )
      return token_res
    return apply


imdb_label_lookup = {0:'negative', 1: 'positive', -1: 'unknown'}

In [8]:
imbd_to_tokens = to_tokens(tokenizer, imdb_label_lookup)

tokenized_dataset = imbd_ds.map(imbd_to_tokens, batched=True, remove_columns=['text', 'label'])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [12]:
training_args = tr.TrainingArguments(
  output_dir="./ut_imdb_movies_classification",
  num_train_epochs =1,
  per_device_train_batch_size=16,
  optim = "adamw_torch",
  report_to =["tensorboard"],
  push_to_hub = True
)

In [13]:
model = tr.AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

data_collator = tr.DataCollatorWithPadding(tokenizer=tokenizer)

trainer = tr.Trainer(
  model,
  training_args,
  train_dataset=tokenized_dataset['train'],
  eval_dataset=tokenized_dataset['test'],
  tokenizer=tokenizer,
  data_collator=data_collator
)

In [14]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.618400
1000,0.138600
1500,0.126100


TrainOutput(global_step=1563, training_loss=0.287262786906725, metrics={'train_runtime': 207.4431, 'train_samples_per_second': 120.515, 'train_steps_per_second': 7.535, 'total_flos': 3383545036800000.0, 'train_loss': 0.287262786906725, 'epoch': 1.0})

In [16]:
fine_tuned_model = tr.AutoModelForSeq2SeqLM.from_pretrained('./ut_imdb_movies_classification')

In [17]:
reviews = [
    """
'Despicable Me' is a cute and funny movie, but the plot is predictable and the characters are not very well-developed. Overall, it's a good movie for kids, but adults might find it a bit boring.""",
    """ 'The Batman' is a dark and gritty take on the Caped Crusader, starring Robert Pattinson as Bruce Wayne. The film is a well-made crime thriller with strong performances and visuals, but it may be too slow-paced and violent for some viewers.
""",
    """
The Phantom Menace is a visually stunning film with some great action sequences, but the plot is slow-paced and the dialogue is often wooden. It is a mixed bag that will appeal to some fans of the Star Wars franchise, but may disappoint others.
""",
    """
I'm not sure if The Matrix and the two sequels were meant to have a tigh consistency but I don't think they quite fit together. They seem to have a reasonably solid arc but the features from the first aren't in the second and third as much, instead the second and third focus more on CGI battles and more visuals. I like them but for different reasons, so if I'm supposed to rate the trilogy I'm not sure what to say.
""",
]

In [19]:
inputs = tokenizer(reviews, return_tensors='pt', truncation= True, padding=True)
predictions = fine_tuned_model.generate(
    input_ids = inputs['input_ids'],
    attention_mask = inputs['attention_mask']
)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [21]:
pdf = pd.DataFrame(
    zip(reviews, tokenizer.batch_decode(predictions, skip_special_tokens=True)),
    columns = ['review', 'classification']
)

display(pdf)

,review,classification
0,"\n'Despicable Me' is a cute and funny movie, b...",negative
1,'The Batman' is a dark and gritty take on the...,positive
2,\nThe Phantom Menace is a visually stunning fi...,positive
3,\nI'm not sure if The Matrix and the two seque...,negative
